In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


In [7]:
from IPython import display
import os
def show_app(app, port = 9999, 
             width = 700, 
             height = 350, 
             offline = False,
            in_binder = None):
    in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ if in_binder is None else in_binder
    if in_binder:
        base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)
        url = 'https://hub.mybinder.org{}'.format(base_prefix)
        app.config.requests_pathname_prefix = base_prefix
    else:
        url = 'http://localhost:%d' % port
        
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          port=port)


In [8]:
app_simple = dash.Dash()

app_simple.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

In [9]:
show_app(app_simple)

Open in new window

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jul/2018 20:43:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2018 20:44:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2018 20:44:00] "GET /_dash-dependencies HTTP/1.1" 200 -


In [10]:
import plotly.graph_objs as go
import plotly.plotly as py
import numpy as np

app_iplotly = dash.Dash()

app_iplotly.layout = html.Div([
    html.Div([
            html.Div(id='click_info'),
             dcc.Graph(id='cur_plot', animate=False)
            ],
             id='plot_div')
])

def _simple_image(xval, yval, as_uri = True):
    print(xval, yval, 'creating')
    lin_x = np.arange(512)
    lin_y = np.arange(512)
    xx, yy = np.meshgrid(lin_y, lin_x)
    out_mat = np.sqrt(np.square(xx-xval)+np.square(yy-yval))
    if as_uri:
        fig, ax1 = plt.matshow(out_mat)
        out_uri = fig_to_uri(fig)
        return dict(source = out_uri,
                    x = lin_x.min(),
                    sizex = lin_x.max()-lin_x.min(),
                    y = lin_y.max(),
                    sizey = lin_y.max() - lin_y.min(),
                    xref= "x",
                    yref= "y",
                    sizing= "stretch",
                    opacity= 0.5,
                    layer= "below"
                   )
    else:
        return out_mat

In [11]:
def update_graph(x_val, y_val):
    traces = [go.Heatmap(z=_simple_image(x_val, y_val, as_uri = False),
                        #colorscale='Viridis'
                        )]
    if (x_val>0) or (y_val>0):
        traces += [go.Scatter(x = [x_val], y = [y_val])]
    
    layout = go.Layout(
        #images= [_simple_image(x_val,y_val)],
        annotations=[
        dict(                            # all "annotation" attributes: /python/reference/#layout-annotations
            text="Current Position ({}, {})".format(x_val, y_val),    # /python/reference/#layout-annotations-text
            x=x_val,                         # /python/reference/#layout-annotations-x
            xref="y",                # /python/reference/#layout-annotations-xref
            y=y_val,                         # /python/reference/#layout-annotations-y
            yref="y"                 # /python/reference/#layout-annotations-yref
        )
    ]
                      )
    return dict(data = traces, layout=layout)


In [12]:
def get_hover_info(hover_data):
    if hover_data is not None:
        clicks = hover_data.get('points', [])
        if len(clicks)>0:
            x, y = clicks[0].get('x', 0), clicks[0].get('y', 0)
            print(x,y, 'hovering')
            return x,y
    return 0,0

@app_iplotly.callback(
    Output(component_id='cur_plot', component_property='figure'),
    [Input(component_id='cur_plot', component_property='clickData')]
)
def update_click_info(hover_data):
    return update_graph(*get_hover_info(hover_data))

In [13]:
show_app(app_iplotly)

Open in new window

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jul/2018 20:50:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2018 20:50:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2018 20:50:16] "GET /_dash-dependencies HTTP/1.1" 200 -


0 0 creating


[2018-07-23 20:50:16,814] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\piush\AppData\Local\conda\conda\envs\viz\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\piush\AppData\Local\conda\conda\envs\viz\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\piush\AppData\Local\conda\conda\envs\viz\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\piush\AppData\Local\conda\conda\envs\viz\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\piush\AppData\Local\conda\conda\envs\viz\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\piush\AppData\Local\conda\conda\envs\viz\lib\site-packages\flask\app.py", line 1598, 

In [14]:
import pandas as pd

df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'c78bf172206ce24f77d6363a2d754b59/raw/'
    'c353e8ef842413cae56ae3920b8fd78468aa4cb2/'
    'usa-agricultural-exports-2011.csv')


app_pd_table = dash.Dash()

app_pd_table.layout = html.Div(children=[
    html.H4(children='US Agriculture Exports (2011)'),
    html.Source(src = df.to_html())
])

In [ ]:
show_app(app_pd_table)

Open in new window

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jul/2018 20:51:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2018 20:51:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2018 20:51:44] "GET /_dash-dependencies HTTP/1.1" 200 -
